# Setup

In [4]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "ann"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)


What if you want to send different subsets of input features through the wide or deep paths? We will send 5 features (features 0 to 4), and 6 through the deep path (features 2 to 7). Note that 3 features will go through both (features 2, 3 and 4). Adding an auxiliary output for regularization:

In [5]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [6]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

In [7]:
np.random.seed(42)
tf.random.set_seed(42)

In [8]:
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")

hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)

concat = keras.layers.concatenate([input_A, hidden2])

output = keras.layers.Dense(1, name="main_output")(concat)

aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)

model = keras.models.Model(inputs=[input_A, input_B],
                           outputs=[output, aux_output])

In [9]:
model.compile(loss = ["mse", "mse"], 
              loss_weights = [0.9, 0.1], 
              optimizer = keras.optimizers.SGD(learning_rate = 1e-3))

In [10]:
history = model.fit([X_train_A, X_train_B], 
                    [y_train, y_train], 
                    epochs = 20,
                    validation_data = ([X_valid_A, X_valid_B], [y_valid, y_valid]))

Epoch 1/20
363/363 [==============================] - 3s 3ms/step - loss: 2.2221 - main_output_loss: 2.0597 - aux_output_loss: 3.6842 - val_loss: 2.7773 - val_main_output_loss: 1.7661 - val_aux_output_loss: 11.8783
Epoch 2/20
363/363 [==============================] - 1s 3ms/step - loss: 0.8646 - main_output_loss: 0.7105 - aux_output_loss: 2.2512 - val_loss: 2.0032 - val_main_output_loss: 0.7552 - val_aux_output_loss: 13.2354
Epoch 3/20
363/363 [==============================] - 1s 3ms/step - loss: 0.7373 - main_output_loss: 0.6240 - aux_output_loss: 1.7578 - val_loss: 1.9543 - val_main_output_loss: 0.8547 - val_aux_output_loss: 11.8499
Epoch 4/20
363/363 [==============================] - 1s 3ms/step - loss: 0.6752 - main_output_loss: 0.5826 - aux_output_loss: 1.5082 - val_loss: 1.5990 - val_main_output_loss: 0.5950 - val_aux_output_loss: 10.6350
Epoch 5/20
363/363 [==============================] - 1s 3ms/step - loss: 0.6315 - main_output_loss: 0.5501 - aux_output_loss: 1.3635 - val_

In [11]:
total_loss, main_loss, aux_loss = model.evaluate(
    [X_test_A, X_test_B], [y_test, y_test])

y_pred_main, y_pred_aux = model.predict([X_new_A, X_new_B])

1/1 [==============================] - 1s 566ms/step
